**IBM DATA SCIENCE PROFESSIONAL CERTIFICATE CAPSTONE PROJECT WEEK 4 JASON MEIL**
This notebook is for the capstone prooject for the 9-course specialization in Data Science from IBM. This project will solve the problem identified in https://github.com/OneeyedCheshire/COursera-Capstone/blob/master/Final_CapStone_Project_Part_1_Loudoun_County.ipynb using Foursquare API and applying data science methodology. 

In [0]:
# Importing all API's and Libraries for Data Analysis
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import folium

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

from sklearn.preprocessing import StandardScaler, normalize, scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score


In [0]:
#Using the following URL https://logis.loudoun.gov/weblogis/ I am going to scrape the website with Beautiful Soup
#to access data on commercial, ag and residential zoning in Loudoun County.
data = requests.get('https://logis.loudoun.gov/weblogis/').text
soup = BeautifulSoup (data, 'html.parser')


In [19]:
pd.read_csv('https://raw.githubusercontent.com/OneeyedCheshire/COursera-Capstone/master/Loudoun_Zoning.csv')


,OBJECTID,ZO_ZONE,ZO_ORDINANCE,ZO_SPEC_CODE,ZO_PROJ_NUM,ZO_UPDATE_NUM,ZO_ZONE_DATE,ZO_ZONE_ORD,ZO_UPD_DATE,agstype,SHAPE_Length,SHAPE_Area,ZO_MAINT_TASK_ID,ZO_GIS_START_DATE,ZO_GIS_END_DATE
0,1,,,NV,NO VALUE,NO VALUE,1899-12-30T00:00:00.000Z,NO VALUE,1899-12-30T00:00:00.000Z,,0.000000e+00,0.000000e+00,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
1,2,A10,1993 R,NaN,ZMAP-1998-0009,Z2000-0001,1900-01-01T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.355895e+04,9.888972e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
2,3,A10,1993 R,NaN,ZMAP-1997-0011,Z2000-0002,1998-06-03T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.523962e+04,1.338334e+07,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
3,4,A10,1993 R,NaN,ZMAP-0000-0293,,1980-05-19T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.049584e+04,5.402650e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
4,5,A10,1993 R,NaN,ZMAP-1995-0011,,1996-09-18T00:00:00.000Z,A10 1993,2015-10-08T00:00:00.000Z,A10,8.483328e+03,2.743747e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
5,6,A10,1993 R,NaN,ZMAP-2000-0004,Z2002-0002,2001-10-15T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.399263e+04,8.409871e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
6,7,A10,1993 R,NaN,ZMAP-1999-0013,Z2001-0005,2000-12-18T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,8.210095e+03,2.774240e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
7,8,A10,1993 R,NaN,ZMAP-1987-0025,,1988-06-20T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,7.569134e+03,3.109151e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
8,9,A10,1993 R,NaN,ZMAP-1985-0010,,1986-04-21T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.534305e+04,9.522835e+06,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z
9,10,A10,1993 R,NaN,ZMAP-1987-0037,,1988-08-01T00:00:00.000Z,A10 1993,2012-04-16T00:00:00.000Z,A10,1.469624e+04,1.311972e+07,,1899-12-30T00:00:00.000Z,1899-12-30T00:00:00.000Z


In [20]:
# Download the geodata
!wget -q -O 'geo.json' https://catalog.data.gov/dataset/loudoun-zoning-8b1c9
print('Data downloaded!')


Data downloaded!


In [0]:
radius = 1000 
limit = 200

venues = []
url = "https://api.foursquare.com/v2/venues/explore?client_id={ZYFKWQQDSLNRXENDGCFHEBONU5LKA1DNK533JE5KK2R5FL3Y}&client_secret={A1ZLCI5XIPEETMVPAGLXXME3ADX3NYYE2PKAMYBVPA3I1TTW}&v={}&ll={},{}&radius={}&limit={}".format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
lat,
long,
radius, 
limit)
  
results = requests.get(url).json

for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

# put the venues into a dataframe
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueType']

y_pred = lreg.predict(X_test)

print('R2-score:', r2_score(y_test, y_pred)) # r2 score
print('Mean Squared Error:', mean_squared_error(y_test, y_pred)) # mse

print('Max positive coefs:', lreg.coef_[np.argsort(-lreg.coef_)[:10]])
print('Venue types with most postive effect:', X.columns[np.argsort(-lreg.coef_)[:10]].values)
print('Max negative coefs:', lreg.coef_[np.argsort(lreg.coef_)[:10]])
print('Venue types with most negative effect:', X.columns[np.argsort(lreg.coef_)[:10]].values)
coef_abs = abs(lreg.coef_)
print('Min coefs:', lreg.coef_[np.argsort(coef_abs)[:10]])
print('Venue types with least effect:', X.columns[np.argsort(coef_abs)[:10]].values)



 

R2-score: 0.273792308888
Mean Squared Error: 0.254179706388
Max positive coefs: [ 0.26348338  0.26213799  0.26213799  0.26213799  0.25818747  0.25818747
  0.25135936  0.24564842  0.23349638  0.22658134]


In [0]:
for i in n_component_list:
    lreg = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(X_pca[:,:i], y, test_size=0.2, random_state=0)
    model = lreg.fit(X_train, y_train)
    # check the result
    y_pred = lreg.predict(X_test)
    r2 = r2_score(y_test, y_pred) # r2 score
    mse = mean_squared_error(y_test, y_pred) # mse
    r2_list.append(r2)
    mse_list.append(mse)
    
scores_df = pd.DataFrame.from_dict(dict([('NComponents', n_component_list),
                                        ('R2', r2_list),
                                        ('MSE', mse_list)]))
scores_df.set_index('NComponents', inplace=True)
In [29]:
# plot the scores to see the best n_components
plt.subplot(1, 3, 1)
scores_df['R2'].plot(kind='line')
plt.title('R2 score / n components')
plt.ylabel('R2 score')
plt.xlabel('n components')

plt.subplot(1, 3, 3)
scores_df['MSE'].plot(kind='line')
plt.title('MSE score / n components')
plt.ylabel('MSE score')
plt.xlabel('n components')

plt.show()

In [30]:
r2_max = scores_df['R2'].idxmax()
print("Best n:", r2_max, "R2 score:", scores_df['R2'][r2_max])

mse_min = scores_df['MSE'].idxmin()
print("Best n:", mse_min, "MSE:", scores_df['MSE'][mse_min])
Best n: 50 R2 score: 0.454460324852
Best n: 50 MSE: 0.190944155714
In [31]:
# Use the best n_components parameter
lreg = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X_pca[:,:r2_max], y, test_size=0.2, random_state=0)
model = lreg.fit(X_train, y_train)

# check the result
y_pred = lreg.predict(X_test)
r2 = r2_score(y_test, y_pred) # r2 score
mse = mean_squared_error(y_test, y_pred) # mse
print("R2 score:", r2)
print("MSE:", mse)


R2 score: 0.454460324852
MSE: 0.190944155714


In [0]:
print('Max positive coefs:', pcr_coefs[np.argsort(-pcr_coefs)[:10]])

print('Max negative coefs:', pcr_coefs[np.argsort(pcr_coefs)[:10]])

coef_abs = abs(pcr_coefs)
print('Min coefs:', pcr_coefs[np.argsort(coef_abs)[:10]])



Max positive coefs: [ 0.07212567  0.0696754   0.06052737  0.0582199   0.05228078  0.05222561
  0.04901431  0.04597368  0.04465698  0.04399769]

Max negative coefs: [-0.05116074 -0.03897274 -0.03710211 -0.03457056 -0.03452567 -0.0345195
 -0.03414522 -0.03304223 -0.03284579 -0.03284275]

Min coefs: [ -8.90366289e-06  -8.90366289e-06   4.09236430e-05  -4.99918920e-05
  -5.87234477e-05   1.27322576e-04   1.27322576e-04   1.27322576e-04
   1.27322576e-04   1.41722883e-04]

